# *Digest functions* SHA256

#### Suplementary functions

In [6]:
def leftrotate(x, c):
    """ Left rotate the number x by c bytes."""
    x &= 0xFFFFFFFF
    return ((x << c) | (x >> (32 - c))) & 0xFFFFFFFF

def rightrotate(x, c):
    """ Right rotate the number x by c bytes."""
    x &= 0xFFFFFFFF
    return ((x >> c) | (x << (32 - c))) & 0xFFFFFFFF

def leftshift(x, c):
    """ Left shift the number x by c bytes."""
    return x << c

def rightshift(x, c):
    """ Right shift the number x by c bytes."""
    return x >> c

# SHA-2 (256)

### Padding


In [7]:
def padding(arg):
        data = bytearray(arg, 'utf-8')
        orig_len_in_bits = (8 * len(data)) & 0xFFFFFFFFFFFFFFFF #
        # 1 add single bit '1' at the end of data
        data.append(0x80)
        # 2. add bits '0' until message length ≡ 448 (mod 512)
        while len(data) % 64 != 56:
            data.append(0)
        # 3. add message length (64 bits) at the end 
        data += orig_len_in_bits.to_bytes(8, byteorder='big')
        assert len(data) % 64 == 0, "Error in padding"
        return data
    
string = 'The quick brown fox jumps over the lazy dog'
print('Message length (B):', len(bytearray(string, 'utf-8')))
print('Message in bits: \n', bin(int.from_bytes(bytearray(string, 'utf-8'), byteorder="big")).strip('0b'))
print('\n')
datapad = padding(string)
print('Padded message: \n', bin(int.from_bytes(datapad, byteorder="big")).strip('0b'))
print('Length of padded message (B):', len(datapad))



Message length (B): 43
Message in bits: 
 1010100011010000110010100100000011100010111010101101001011000110110101100100000011000100111001001101111011101110110111000100000011001100110111101111000001000000110101001110101011011010111000001110011001000000110111101110110011001010111001000100000011101000110100001100101001000000110110001100001011110100111100100100000011001000110111101100111


Padded message: 
 1010100011010000110010100100000011100010111010101101001011000110110101100100000011000100111001001101111011101110110111000100000011001100110111101111000001000000110101001110101011011010111000001110011001000000110111101110110011001010111001000100000011101000110100001100101001000000110110001100001011110100111100100100000011001000110111101100111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000101011
Length of padded message (B): 64


### Extending message 
512 bits = 16 x 32-bits pieces. SHA256 performs 64 round functions. Additional 48 pieces must be generated.

In [8]:
def formchunk(data, offset):
    chunks = data[offset : offset + 64]
    w = [0 for i in range(64)] 
    # 1 16 x 32-bits
    for i in range(16): #0...15 
        w[i] = int.from_bytes(chunks[4*i : 4*i + 4], byteorder='big')
        print(bin(w[i])[2:].zfill(32))
    #2 generate additional 48 pieces
    for i in range(16, 64):
        s0 = (rightrotate(w[i-15], 7) ^ rightrotate(w[i-15], 18) ^ rightshift(w[i-15], 3)) & 0xFFFFFFFF
        s1 = (rightrotate(w[i-2], 17) ^ rightrotate(w[i-2], 19) ^ rightshift(w[i-2], 10)) & 0xFFFFFFFF
        w[i] = (w[i-16] + s0 + w[i-7] + s1) & 0xFFFFFFFF
        print(bin(w[i])[2:].zfill(32))
    return w
        
string = 'The quick brown fox jumps over the lazy dog'
datapad = padding(string)
exdatapad = formchunk(datapad,0)
print('Extended chunk (64x32)\n', exdatapad)


01010100011010000110010100100000
01110001011101010110100101100011
01101011001000000110001001110010
01101111011101110110111000100000
01100110011011110111100000100000
01101010011101010110110101110000
01110011001000000110111101110110
01100101011100100010000001110100
01101000011001010010000001101100
01100001011110100111100100100000
01100100011011110110011110000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000101011000
01001000011101110111100111100011
11000111100110101010011100100101
01010101101110000100110000100010
00001000110010000101010000100110
01001100101101011110010010000001
10000100011010110011011011010000
00101110010111011100110010010101
01001001010001000101011100100000
01100010011110010001000010101111
00101110001111100110110100000101
00010111100100100111111011011101
01000111110111111010000111000100
11110101000111000000010001110100
01010011001010010010100110010001
0111110000

### Round function of SHA256

#### Nonlinear functions 
1. Ch: $(e \wedge f) \oplus (\neg e \wedge g)$ 
2. Ma: $(a \wedge b) \oplus (a \wedge c)  \oplus (b \wedge c)$
3. $\sum_0$: $a\ggg 2 \wedge a\ggg 13 \wedge a\ggg 22$ 
4. $\sum_1$: $e\ggg 6 \wedge e\ggg 11 \wedge e\ggg 25$

In [9]:
# IV vector:
h0 = 0x6a09e667
h1 = 0xbb67ae85
h2 = 0x3c6ef372
h3 = 0xa54ff53a
h4 = 0x510e527f
h5 = 0x9b05688c
h6 = 0x1f83d9ab
h7 = 0x5be0cd19
        
# rounds constants:
k = [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
     0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
     0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
     0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
     0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
     0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
     0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
     0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]    

def sha2round(i, w, a, b, c, d, e, f, g, h):
    S1 = (rightrotate(e, 6) ^ rightrotate(e, 11) ^ rightrotate(e, 25)) & 0xFFFFFFFF
    ch = ((e & f) ^ ((~e) & g)) & 0xFFFFFFFF
    temp1 = (h + S1 + ch + k[i] + w[i]) & 0xFFFFFFFF
    S0 = (rightrotate(a, 2) ^ rightrotate(a, 13) ^ rightrotate(a, 22)) & 0xFFFFFFFF
    maj = ((a & b) ^ (a & c) ^ (b & c)) & 0xFFFFFFFF
    temp2 = (S0 + maj) & 0xFFFFFFFF

    new_a = (temp1 + temp2) & 0xFFFFFFFF
    new_e = (d + temp1) & 0xFFFFFFFF
    # Rotate the 8 variables
    new_a, a, b, c, new_e, e, f, g
    return new_a, a, b, c, new_e, e, f, g

print(h0, h1, h2, h3, h4, h5, h6, h7) 
a, b, c, d, e, f, g, h = h0, h1, h2, h3, h4, h5, h6, h7 
a, b, c, d, e, f, g, h = sha2round(0, exdatapad,a, b, c, d, e, f, g, h)
a, b, c, d, e, f, g, h = sha2round(1, exdatapad,a, b, c, d, e, f, g, h)
a, b, c, d, e, f, g, h = sha2round(2, exdatapad,a, b, c, d, e, f, g, h)


print(a, b, c, d, e, f, g, h )

h0 = (h0 + a) & 0xFFFFFFFF
h1 = (h1 + b) & 0xFFFFFFFF
h2 = (h2 + c) & 0xFFFFFFFF
h3 = (h3 + d) & 0xFFFFFFFF
h4 = (h4 + e) & 0xFFFFFFFF
h5 = (h5 + f) & 0xFFFFFFFF
h6 = (h6 + g) & 0xFFFFFFFF
h7 = (h7 + h) & 0xFFFFFFFF
        # 3. Conclusion
hash_pieces = [h0, h1, h2, h3, h4, h5, h6, h7]

1779033703 3144134277 1013904242 2773480762 1359893119 2600822924 528734635 1541459225
652397644 941978704 1349578093 1779033703 818854285 4251937728 3979364290 1359893119


## Complete SHA256 digest function
1. Add padding 
2. For each 512-bits chunk do: 
    1. split into 16 x 32-bits pieces
    2. generate 48 additional pieces 
    3. for 64 32-bits pieces do round function (add round constant)
    3. add result to digest vector (h0...h7)
4. h0...h7 finally contain digest 

In [15]:
def simpleSHA256(arg):

    h0 = 0x6a09e667
    h1 = 0xbb67ae85
    h2 = 0x3c6ef372
    h3 = 0xa54ff53a
    h4 = 0x510e527f
    h5 = 0x9b05688c
    h6 = 0x1f83d9ab
    h7 = 0x5be0cd19

    k = [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
     0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
     0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
     0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
     0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
     0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
     0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
     0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]    

    #1. Do padding
    datapad = padding(arg)

    #2. For each 512-bits 
    a, b, c, d, e, f, g, h = h0, h1, h2, h3, h4, h5, h6, h7
    for offset in range(0, len(arg), 64):
        # Split into 16x32 bits pieces & add 48 pieces
        chunk = formchunk(datapad,offset)
        a, b, c, d, e, f, g, h = h0, h1, h2, h3, h4, h5, h6, h7
        
        for i in range(64):
            a, b, c, d, e, f, g, h = sha2round(i, chunk, a, b, c, d, e, f, g, h);
            
        h0 = (h0+a) & 0xFFFFFFFF
        h1 = (h0+b) & 0xFFFFFFFF
        h2 = (h0+c) & 0xFFFFFFFF
        h3 = (h0+d) & 0xFFFFFFFF
        h4 = (h0+e) & 0xFFFFFFFF
        h5 = (h0+f) & 0xFFFFFFFF
        h6 = (h0+g) & 0xFFFFFFFF
        h7 = (h0+h) & 0xFFFFFFFF
    
    return h0, h1, h2, h3, h4, h5, h6, h7 
    
    
H0, H1, H2, H3, H4, H5, H6, H7 = simpleSHA256("The quick brown fox jumps over the lazy dog")
hash_pieces = [H0, H1, H2, H3, H4, H5, H6, H7]
print(hash_pieces)
print(sum(leftshift(x, 32 * i) for i, x in enumerate(hash_pieces[::-1])))
digest = sum(leftshift(x, 32 * i) for i, x in enumerate(hash_pieces[::-1]))
raw = digest.to_bytes(32, 'big')
format_str = '{:0' + str(2 * 32) + 'x}'
hash = format_str.format(int.from_bytes(raw, byteorder='big'))
print("Message digest : ", hash)
#3618175923 131563668 1774885564 2953326159 2371244516 1832704886 755159231 935978386

01010100011010000110010100100000
01110001011101010110100101100011
01101011001000000110001001110010
01101111011101110110111000100000
01100110011011110111100000100000
01101010011101010110110101110000
01110011001000000110111101110110
01100101011100100010000001110100
01101000011001010010000001101100
01100001011110100111100100100000
01100100011011110110011110000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000101011000
01001000011101110111100111100011
11000111100110101010011100100101
01010101101110000100110000100010
00001000110010000101010000100110
01001100101101011110010010000001
10000100011010110011011011010000
00101110010111011100110010010101
01001001010001000101011100100000
01100010011110010001000010101111
00101110001111100110110100000101
00010111100100100111111011011101
01000111110111111010000111000100
11110101000111000000010001110100
01010011001010010010100110010001
0111110000

[1416127776, 1903520099, 1797284466, 1870097952, 1718581280, 1786080624, 1931505526, 1701978228, 1751457900, 1635416352, 1685022592, 0, 0, 0, 0, 344, 1215789539, 3348801317, 1438141474, 147346470, 1286988929, 2221618896, 777899157, 1229215520, 1652101295, 775843077, 395476701, 1205838276, 4112254068, 1395206545, 2082732661, 642658196, 4090892447, 242423513, 3220830233, 1506285376, 3744511548, 1721749689, 4278994071, 1712105681, 2964186611, 3607998669, 667645411, 2343530444, 2460417689, 3360953683, 709641788, 82325619, 3978005588, 2935752878, 2443156070, 3249509347, 1880188090, 3991173587, 654621989, 2400793299, 375831977, 1200352507, 3329280995, 646706396, 3105249477, 3773244089, 2336341666, 3891224983]

## Reference implementation from haslib library

In [13]:
import hashlib 
h = hashlib.sha256()
h.update(b'The quick brown fox jumps over the lazy dog')
h.hexdigest()

'd7a8fbb307d7809469ca9abcb0082e4f8d5651e46d3cdb762d02d0bf37c9e592'